In [3]:
import psycopg2
import geopandas as gpd
from pathlib import Path
import json

In [4]:
credentials = Path('../../inputs/db_credentials.json')
with open(credentials, 'r') as f:
    credentials = json.load(f)

user     = credentials['user']
password = credentials['password']
host     = credentials['host']
port     = credentials['port']
database = credentials['database']

In [15]:
# query = f'SELECT ST_Transform(ST_Buffer(ST_UNION(geom), 0), {SRID}) as geom FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# query = f'SELECT ST_Transform(ST_ExteriorRing(ST_UNION(geom)), {SRID}) as geom FROM thuc_cell_geo({lat}, {lon}, thuc_near_run_id({lon},{lat}))' ;


# cur.execute("SELECT table_name FROM information_schema.tables")

# query = f'SELECT ST_AsGeoJSON(ST_Transform(ST_UNION(ST_ExteriorRing(geom)), {SRID})) FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;

# query = f'SELECT geom FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# query = f'SELECT ST_Transform(ST_UNION(ST_ExteriorRing(geom)), {SRID}) FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;


# query = f'SELECT thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE'"

In [22]:
# conn = psycopg2.connect(dbname='THUC', user='dbadmin', password='vxn&584dkj+%38#3aj!', host="130.74.75.55", port="5432")
conn = psycopg2.connect(database=database,
                        host=host,
                        user=user,
                        password=password,
                        port=port)
cur = conn.cursor()

SRID = 4326
lon, lat = -89.9078220, 34.2385274

# query = f'SELECT ST_AsGeoJSON(ST_UNION(ST_Transform(ST_ExteriorRing(geom), {SRID}))) FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# boundary_sql = f"SELECT ST_AsGeoJSON(ST_Transform(ST_UNION(geom), {SRID})) FROM thuc_cell_geo({lon},{lat}, thuc_near_run_id({lon},{lat}))"
# cells_sql = f"SELECT ST_AsGeoJSON(ST_Transform(ST_UNION(ST_ExteriorRing(geom)), {SRID})) FROM thuc_cell_geo({lon},{lat},thuc_near_run_id({lon},{lat}))"


# simple_cells = "SELECT * FROM thuc_1002_annagnps_cell_ids LIMIT 10"

# query = simple_cells
query = """
ALTER TABLE thuc_1004_annagnps_cell_data_section
ALTER COLUMN mgmt_field_id TYPE TEXT;
"""

cur.execute(query)

# tables = cur.fetchall()

# for table in tables:
#     print(table[0])

# cur.close()
# conn.close()

In [13]:
from sqlalchemy import URL, create_engine, text as sql_text
import geopandas as gpd

# create a SQLAlchemy engine object
url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

engine = create_engine(url_object)

# query = "SELECT * FROM thuc_1002_annagnps_cell_ids LIMIT 5"
query = "SELECT * FROM thuc_1148_annagnps_cell_ids LIMIT 5"
# query = "SELECT * FROM thuc_1002_annagnps_cell_data_section ORDER BY cell_id"# WHERE cell_id=22"

gdf = gpd.read_postgis(sql=sql_text(query), con=engine.connect(), geom_col='geom')
# gdf = gpd.read_postgis(sql=sql_text(query), con=conn, geom_col='geom')
gdf

,fid,dn,geom
0,1,664181,"POLYGON ((790248.788 3877340.539, 790248.788 3..."
1,2,664131,"POLYGON ((790188.788 3877160.539, 790188.788 3..."
2,3,664211,"POLYGON ((791148.788 3877070.539, 791148.788 3..."
3,4,671441,"POLYGON ((813558.788 3877070.539, 813558.788 3..."
4,5,664121,"POLYGON ((789288.788 3876800.539, 789288.788 3..."


In [15]:
import pandas as pd
from sqlalchemy import URL, create_engine, text as sql_text

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

engine = create_engine(url_object)

# query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'
query = "SELECT * FROM thuc_0917_annagnps_cell_data_section ORDER BY cell_id"
# query = """
# SELECT 
#    table_name, 
#    column_name, 
#    data_type 
# FROM 
#    information_schema.columns
# WHERE 
#    table_name = 'thuc_0917_annagnps_cell_data_section';
# """

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df.head()

,cell_id,soil_id,mgmt_field_id,reach_id,reach_location_code,cell_area,time_of_conc,avg_elevation,rcn_calib_id,avg_land_slope,...,shallow_conc_flow_length,delivery_ratio,constant_usle_c_fctr,constant_usle_p_fctr,all_oc_calib_fctr,all_n_calib_fctr,all_p_calib_fctr,sheet_and_rill_erosion_calib_fctr,gullies_erosion_calib_fctr,input_units_code
0,22,294151,None,2,1,0.27,None,75.23,None,0.00681,...,0.0,None,None,None,None,None,None,None,None,1
1,32,294152,None,3,1,45.63,None,83.74,None,0.06104,...,50.0,None,None,None,None,None,None,None,None,1
2,33,294129,Mixed_Forest,3,1,31.32,None,84.52,None,0.06458,...,50.0,None,None,None,None,None,None,None,None,1
3,41,294142,None,4,0,11.97,None,95.08,None,0.03059,...,50.0,None,None,None,None,None,None,None,None,1
4,42,294142,None,4,1,6.30,None,90.31,None,0.02815,...,50.0,None,None,None,None,None,None,None,None,1


In [13]:
from sqlalchemy import URL, create_engine, text as sql_text

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

engine = create_engine(url_object)

connection = engine.connect()

try:
    query = """ ALTER TABLE thuc_1004_annagnps_cell_data_section ALTER COLUMN mgmt_field_id TYPE TEXT """
    connection.execute(sql_text(query))

    # Commit the transaction explicitly
    connection.commit()

except Exception as e:
    # Rollback the transaction in case of an error
    connection.rollback()

    # Optionally, you can print or log the error message
    print("Error:", str(e))

finally:
    # Close the connection
    connection.close()